## 一、Tomcat优化思路梳理
* 无限制的增大业务线程。
* 优化请求队列的数量。
* 第一，根据服务器情况，来制定业务线程池的数量和请求队列的数量；第二，如果业务线程和请求队列一定的情况下，让我们的业务线程处理的更快；第三，优化线程，思路:a.加大它的资源（内存更多或CPU更多），b.对线程模型的优化。

## Connector配置建议
### 线程池优化
* `maxConnections`：最大连接数。
    1. 最大连接数是受系统内核的影响。`Linux`的系统内核数，可以通过`ulimit －a`的方式查看服务器上允许的最大连接数。修改`/etc/security/limits.conf`下，把`core`和`rss`两项进行放开，把这两个都变成`nofiles`，且对应的值都是**65535**。
    2. 添加maxConnections的配置。（配置完成以后，一定要进行压测！超过Tomcat最大承载能力的时候，性能会出现急剧降低的现象。）
        2.1 **对CPU要求更高时**，建议不要配置过于大；
        2.2 **对CPU要求不是特别高时**，建议配置在**3000左右**。
    3. 配置的地方:`server.xml`中找`connector`。
* `maxThreads`：最大线程数。服务器上**500－700是相对合理的数量**。配置的位置也是在`server.xml中的connector`里面进行配置。
* `acceptCount`:最大排队等待数（也就是前面我们提到的请求队列）。**（单Tomcat能处理的最大数量就是`maxThreads`与`acceptCount`的数量之和）**。`一般建议与最大线程数持平或略低就可以接受。`

### Tomcat内存优化（能使每一个连接处理的速度更快）
JVM优化建议

参数|参数作用|优化建议
:-:|:-:|:-:
-server|启用Serveer|服务端建议开启
－Xms|最小内存|建议与－Xmx相同
－Xmx|最大内存|建议到可用内存的80%
－XX:MetaspaceSize|元空间初始值|
-XX:MaxMetaspaceSize|元空间最大内存|默认无限
-XX:MaxNewSize|新生代最大内存|默认16M

* JVM在最大内存和最小内存进行漂移的过程中，本身是浪费资源的。当最大内存和最小内存配置一样的时候，JVM就不会来回漂移。
* 上面的第4和第5是专门针对1.8及以上的。（以前是持久内存）
* 修改的位置在`tomcat/bin/catalina.sh`。通过添加`JAVA_OPTS="-server -Xms128m -Xmx128m -XX:MetaspaceSize=128m -XX:MaxMetaspaceSize=128m -XX:MaxNewSize=32m"`
* 验证的方式是通过启动后，通过`jmap -heap [进程编号]`的命令进行查看。

### Tomcat其他内容优化
* Gzip相关配置(**修改的位置在conf/server.xml中的Connector上**)
    * `compression`:设置开启Gzip压缩(True或者False)。
    * `compressableMimeType`:压缩类型()。
    * `compressMinSize`:压缩后输出内容大小(2048等具体的值)。
* 其他项配置
    * `enableLookups`:开启反查域名(某些场景下比较好用，但是一般还是不用开启)。
    * `connectionTimeout`:网络连接超时阈值。
    * `minSpareThreads`:最小空闲线程数(**没有访问的情况下保持多少活跃**)。

### Tomcat三种线程模式介绍
1. **BIO：**最稳定最老的一个连接器，使用阻塞形式处理Request请求。
2. **NIO：**使用后Java的异步IO技术，进行非阻塞形式处理Request请求。
3. **APR：**原生C语音编写的非阻塞I/O，目前性能最理想。

### Apr安装(3个内容都要下载，除此之外，**还需要通过yum安装`expat`和`expat-devel`两个依赖源**。)
#### apr安装步骤：
1. apr：解压至安装目录；`./configure -prefix=/usr/local/apr[安装的目录]`；`make`；`make install`；
2. apr-iconv：解压至安装目录；`./configure -prefix=/usr/local/apr-iconv[自己的安装目录] --with-apr=/usr/local/apr[apr的安装目录]`;`make`;`make install`。
3. apr-util：解压至安装目录；`./configure -prefix=/usr/local/apr-util[自己的安装目录] --with-apr=/usr/local/apr[apr的安装目录] --with-apriconv=/usr/local/apr-iconv[apr-iconv/bin/apriconv的安装目录]`;`make`;`make install`。

### Tomcat的apr配置
1. 到`Tomcat`下的`bin`目录下
2. 解压`tomcat-native.tar.gz`文件，然后进入当前解压生成的目录
3. 然后进入`native`目录进行安装，通过`./config --with-apr=/usr/local/apr[自己的apr的安装目录]`。最后通过`make&make install`进行安装。
4. 修改`catalina.sh`，在其中的`JAVA_OPTS`那一行的下面添加`LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/apr/lib export LD_LIBRARY_PATH`就把apr的影响范围限制到比较小的范围。
5. 修改`server.xml`文件。修改`Connector`中的`protocl`这个协议。

### Tomcat集群方式：
1. Apache Tomcat Clustering(需要验证session等内容，做session的同步和传输，对比耗时，网络传输耗时)；
    1.1 去Tomcat的官网，找到`Clustering`内容的部分，找到相关需要配置的内容；
    1.2 去修改`server.xml`文件，在`Engine`这个节点下进行内容的添加。（如果需要修改的话，也就是改改`address`和`port`标签的内容，因为实际上是内网的交互，实际上不改也没问题。）
    1.3 对于那些需要在集群中同步数据的项目，需要在`web.xml`文件中，添加`<distributable/>`标签(这样就能解决集群中绝大多数的问题)。
2. JWT等类似机制(实际上是无状态的)；
3. MSM(Memcached-Session-Manager，把session交给`Memcached`进行管理，这种方式的特点是`Memcached`在内存的利用方面无人能及，**比redis性能提升3%-5%**)、Tomcat+Redis(**具有持久化，和多种多样的数据形式**)等统一Session管理；

> 我们可以通过安装一个小软件来实现文件的上传和下载。这个是安装在`centos`系统中的，可以通过`yum -y install lrzsz`来进行安装。之后可以直接通过`rz`命令进行上传。(**但是在网络服务器中，这种并不好使！！**)

# 三、Memcached基础及底层机制剖析
## 1. Memcached入门
### 1.1 Memcached是啥？
> Memcached概念
> * `Memcached`是一款高性能、分布式的内存对象`缓存系统`；
> * `Memcached`可以有效分担数据库负载；
> * `Memcached`基于`libevent`事件处理实现无阻塞通信；
### 1.2 Memcached能干啥？

### 1.3 Memcached安装部署
> 安装需要两个包，`libevent(libevent.org)`包和`memcached(memcached.org)`的包。
> 先安装`libevent`:1. 解压缩；2. `./configure --prefix=/opt/install/libevent[我们的安装目录]`；3. `make`；4. `make install`。
> 再安装`memcached`：1. 解压缩；2.`./configure --prefix=/opt/install/memcached[我们的安装目录] --with-libevent=/opt/install/libevent[libevent的安装目录]`；3.`make`；4.`make install`。
### 1.4 Memcached启动参数
参数|参数说明
:----------|:----------
-d|启动一个守护进程
-m|分配给Memcached使用的内存数量(单位是M)
-u|运行Memcached的用户
-l|监听的服务器IP地址
-p|监听的端口
-c|最大运行的并发连接数
-P|设置保存Memcache的pid文件

## 2. 基础命令与原理
命令|命令作用|特殊说明
:---------|:-----------------------| :-------------
set|用于向缓存添加新的键值对|无论什么情况，都可以插入
add|当缓存中不存在键时则写入|只有当key不存在的情况下，才可以插入
replace|当键已经存在时进行替换| 替换，1.只修改已存在key的value值；2.如果key不存在，则不会进行操作。
append|在已有结果上追加数据| 追加后面内容。 1.最后的length参数实际上是我们追加的值的长度，而不是总长度；2.如果key不存在，则不会进行
prepend|在已有数据前补充数据| 追加头内容。1.最后的length参数实际上是我们追加的值的长度，而不是总长度；2.如果key不存在，则不会进行
cas|检查和更新，通常与gets一起使用
get/gets|获取数据/数据+版本号
delete|删除数据| 1.delete操作并不会真正删除数据；2.将数据打一个删除标记。
incr/decr|增加/减少数值| 1.只能操作能转换为数字的Value；2.desr不能将数字减少至0以下。

Memcached本身并没有提供命令行工具，我们可以通过`yum install -y telnet`来安装一个最简单的命令行工具。安装完成之后，可以通过`telnet 192.168.1.18[我们的ip地址] 2222[我们的端口]`来连接Memcached。

操作|命令
 :------ | :-------------------------- 
新增操作| set/add kl 0 0 2，输入之后，键入value值
修改操作| replace/append/prepend(这些是直接修改原来的值) key flag exTime length
查询操作| get key
检查更新| cas(Check And Set。流程：1. 输入待修改的数据+版本号；2. Memcached检测版本号是否正确；3.如果正确，则修改数据) key flag exTime length version -> value/gets key(首行是：key 持久化时间 长度 版本号)
删除操作| delete key
增减操作| incr/decr key value
flush_all| flush_all,不会释放内存，但会清掉所有数据

> 注意事项：
> 1. Memcached是key/value键值对形式存储
> 2. key的样子： key flags exTime(过期的时间) length(value的长度)

### Memcache内存分配机制(Slab Allocator[分配器])
1. Slab(内存容器，同一个Slab下的Chunk的大小是一样的):
2. Page(小版本的内存容器，默认大小是1M):
3. Chunk(真实的存储数据的“保险箱”。需要注意的是：1、Chunk是预分配大小的；2、在未定义的情况下，Chunk的大小是80Byte；3、不同Slab的Chunk大小不一样；4. 相同Slab的Chunk大小固定；5.宁可内存不整除被浪费，Chunk大小也不会变。):
4. 自增长因子(默认情况下是**1.25**):
5. Slab Class(1.存储Slab的元数据，如Chunk的大小，2. 存储样式。从而把需要放入的Chunk数据放到最合适的里面):

### 寻找Chunk的经历
1. Slot【垃圾桶，讲师的称呼】：<br/>
    1.1 delete后，将Chunk标识放入Slot;
    1.2 数据过期，将Chunk标识放入Slot；
2. 寻找的过程：<br/>
    2.1 在Slot中寻找不用的Chunk；
    2.2 使用空闲的Chunk；
    2.3 触发LRU(最近最少使用)流程；
3. 注意事项：<br/>
    3.1 **不会因为其他Slab空闲，就不触发LRU流程**。

## 3. Memcached与Java结合
> 注意事项：
> 1. Memcached是C语言编写的；
> 2. Memcached提供了诸多语言的客户端；

### Java客户端
1. java_memcached:很旧，官方的，基于`传统IO`。
2. spymemcached:市面上也比较常用。
3. **xmemcached**:查询处理方面，效率更高。

### 深入使用XMemcached
#### 1. 构建MemcachedClient对象
#### 2. 更新过期时间之`Touch`演示
1. 注意事项：<br/>
    1.1 创建`MemcachedClient`是通过`MemcacgedClientBuilder`来进行创建和配置的，**这样不仅可以使用文本协议，还可以使用二进制协议**。<br/>
`touch`是用来**更新过期时间的**。
#### 3. XMemcached的cas操作：
有自己的cas操作，不过是不同的api，除了传key之外，还需要提供回调函数(`getMaxTries`是重试次数；`getNewValue`是修改内容，第一个参数是当前的版本，第二个参数是目前的值，这个方法的返回值是你要修改的值)。

#### 4.命名空间操作
`client`通过`withNamespace`方法来进行，其中第一个参数是`命名空间的名称`,第二个参数是回调方法(`MemcachedClientCallable`,里面有`call`方法里面进行普通的操作！)

### 初探Memchahed分布式原理
1. Memcached的分布式跟特殊:**客户端分布式**
2. Memcached每一个服务器端都不相互通信。
3. Memcached客户端通过**算法**保证数据的唯一性。

### XMemcached整合SpringBoot

### 讲解XMemcachedBuilder相关参数
命令|命令作用|注意事项
:----|:----------|-------------------:
authInfoMap|配置身份验证信息|可能配置了多个节点，每个节点单独配置
bufferAllocator|设置NIO ByteBuffer适配器|默认就很好用，一般不用改
**commandFactory**|设置协议模式，默认是文本协议|一般是会配置成二进制的协议，传输更小，更快，**但是Memcached中有些东西必须是文本协议，二进制协议是搞不定的**
configuration|设置与Session有关的诸多参数|是个对象，可以配置很多相关的配置
**connectionPoolSize**|设置NIO连接池链接数量
connectionTimeout|设置超时时间
enableHealSession|是否开启健康检查
failureModel|是否开启failure模式，**默认是false**|如果开启了，这个节点失败了，就不会去寻找下个节点，直接报错；如果不开启就会查找下个节点
healSessionInterval|健康检查间隔，**默认是2秒**
keyProvider|Key转换器，是否进行转义|是策略模式的体现，是个**接口**，但是很少用到。可以对key进行自定义。或者对key进行统一的处理。
maxQueueNoReplyOperations|设置最爱的noreply数量
name|设置缓存实例名
**opTimeout**|设置默认**全局操作**超时时间
sanitizeKeys|开启/关闭对URL进行encode|如果key是个url，会对url进行转换，来减少空间。
**sessionLocator**|设置客户端Hash方式|除了`余数Hash`和`一致性Hash`，还提供了**选举Hash**，声称可以在一定程度上替代`一致性Hash`，但是使用来看效果并不理想。
socketOption|设置TCP访问的相关参数
stateListeners|设置state的监听器|一般不会在服务器上开启，会很大的耗费资源。服务器上会用一些其他的工具进行操作。
transcoder|设置序列转换器|默认是Serializeble的转换器。

## 4. Memcached集群
在使用XMemcached的时候，在`MemcachedClient`中在地址里面通过**空格**来分隔不同的memcached。
#### 1. 注意：
    1.1. 自身通过算法保障数据唯一性。
    1.2. 集群形式对用户和Memcached都是透明的。
    1.3. Memcached的集群是通过客户端实现的。
    1.4. Memcached服务端相互不认识。
#### 2. 常见的分布式算法：
    2.1 余数Hash(这个过程用在了所有的操作中)
        2.1.1 过程
            2.1.1.1 将传入的key转换为Hash值
            2.1.1.2 获取服务器数量列表
            2.1.1.3 Hash值%服务器数量
            2.1.1.4 通过余数来选择具体存放/查询的服务器
        2.1.2 优缺点：
            2.1.2.1 简便易理解(优点)
            2.1.2.2 增加/减少节点，会造成“灾难”
    2.2 一致性Hash
        2.2.1 过程
            2.2.1.1 将服务器列表+虚拟节点分布在0-2的32次方的一个圆上。
            2.2.1.2 将传入的key转换为Hash值。
            2.2.1.3 Hash值%服务器数量
            2.2.1.4 通过余数来选择具体存放/查询的服务器
            2.2.1.5 如果余数命中虚拟节点，则会顺时针寻找真实Memcached服务。
        2.2.2 优缺点(实际上刚好与余数Hash的优缺点相反)
    2.3 面试题：
        2.3.1 Memcached的两段Hash
            2.3.1.1 客户端通过Hash算法寻找存储/查询节点
            2.3.1.2 进入Memcached以后，通过Hash算法，寻找具体Chunk。

## 5. 使用场景与优化建议

### Memcached调优思路
#### 1.提高内存命中率
#### 2.减少内存浪费
#### 3.增加内存的重复利用率

### Memcached辅助调优命令
#### 5.1 基础命令
1. `Stats`命令：查看服务器的**运行状态**和**内部数据**。
2. `Stats settings`:查看服务器设置。
3. `Stats items/slabs`:数据项统计/区块统计。

#### 5.2 缓存命中率
1. 相关参数
    1.1 cmd_get（获取所有的请求次数，返回的次数等于下面两个次数的和，通常拿这三个参数做命中率的分析）
    1.2 get_hits（查询的时候，总的命中内存的次数）
    1.3 get_misses（查询的时候，未查到的次数）
2. 计算公式：(get_hits/cmd_get)*100%

#### 5.3 分析对象LRU频率
1.相关参数：
    1.1 curr_items（当前存储的条数）
    1.2 total_items(从系统开始运行，总共存储的数据条数)
    1.3 evictions(是删除的items的数量，也就是我们打了删除标记的items的数量，这些 数量不会出现在上面两个中。这个东西是在`Lru`的时候才会触发改变！！)

#### 5.4 分析字节数流量
1. 相关参数
    1.1 bytes（我们现在的存储的数据量）
    1.2 bytes_read(总共读了多大的数据量)
    1.3 bytes_written(我们的总的写入数量)
2. 注意事项：
    2.1 理论上`bytes_read`的数值越大越好，这样我们就是在合理的使用，否则就是要从数据库里查。
    2.2 `bytes_written`的值主要用于运维，看内存是否足够。以及分析对数据库分担了多少压力。(包含key，value，flag以及内定义的一些级别)

#### 5.5 分析CPU占用情况
1. 相关参数
    1.1 rusage_user(占用的系统用户时间，**当前的进程执行了多少时间**)
    1.2 rusage_system(占用的系统时间，**占用的CPU的时间**)
    
#### 5.6 分析连接情况
1. 相关参数
    1.1 curr_connections(当前有多少连接)
    1.2 total_connections(总共有多少连接)

#### 5.7 stats settings核心参数
参数|参数作用|说明
:------|:---------------|:-------
maxbytes|最大字节数限制，0表示无限制|在启动时候的设置参数
maxconns|允许最大连接数
growth_factor|自增长因子
chunk_size|key+value|flags大小
reqs_per_event|最大IO吞吐量

#### 5.8 stats items核心参数
参数|参数作用|说明
:------|:--------------|:----------
number|该slab中对象数，不包含过期对象
age|LRU队列中最老对象的过期时间
evicted|LRU释放对象数
evicted_nonzero|设置了非0时间的LRU释放对象数
evicted_time|最后一次LRU描述，监控频率
outofmemory|不能存储对象数
tailrepairs|修复slabs次数
reclaimed|使用过期对象空间存储对象次数

#### 5.9 stats slab核心参数
参数|参数作用|说明
:------|:--------------|:---------
chunk_size|该slab中对象数，不包含过期对象
chunk_per_page|LRU队列中最老对象的过期时间
total_pages|LRU释放对象数
total_chunks|设置了非0时间的LRU释放对象数
get_hits|最后一次LRU秒数，监控频率
userd_chunks|不能存储对象次数
free_chunks|修复slabs次数
mem_requested|使用过期对象空间存储对象次数
active_slabs|slab数量

### Slab Allocator效果演示
1. items的大小包含下面几部分：<br/>
    1.1 key<br/>
    1.2 value<br/>
    1.3 flags<br/>
    1.4 数据结构（**32字节**，用于帮助Memcached来完成自身管理上的东西）<br/>
    1.5 suffix（**16~17字节之间**，是Memcached的自身管理的机制，我们不能进行设定）<br/>
    1.6 空间（**10字节**）<br/>
2. 延迟过期（设置了个时间戳，在查询的时候进行判断）

### 内存调优建议、目标和常见问题
#### 1 内存调优：
1. 问题：
    1.1 存不满Chunk；
    1.2 热点数据的堆积；
    1.3 Slab不能被page整除；
    1.4 Page不能比Chunk整除；
2. 思路：
    2.1 调整Chunk大小；
    2.2 调整自增长因子；
3. 场景分析：
    3.1 MSM
        3.1.1 特点：
            3.1.1.1 数据长度集中在某几个区域内；
            3.1.1.2 非均匀分布
        3.1.2 思路
            3.1.2.1 分析集中在哪几个区域；
            3.1.2.2 能不能将Chunk调整成一致大小；
            3.1.2.3 调整Chunk大小和自增长因子，slab大小
    3.2 等长数据
        3.2.1 特点：
            3.2.1.1 数据长度集中在一个区域内
            3.2.1.2 很极端非均匀分布
        3.2.2 思路：
            3.2.2.1 分析集中在哪几个区域；
            3.2.2.2 将Chunk调整成一致大小。
        3.2.3 方案：
            3.2.3.1 将Chunk调整成与业务数据长度保持一致，或略有冗余；
            3.2.3.2 调整自增长因子为**1.01**（**不支持1.0**）。
            
#### 2 Memcached使用限制
1. **不提供持久化机制（虽然可以通过导入与导出，但是不建议）**；
2. Memcached只有理论上的永久持久化**30天**（只有在Memcached没有down掉的情况下），通过常量`REALTIME_MAXDELTA`设置（这个在Memcached的源码里面，需要对Memcached进行二次开发）；
3. **不提供安全管理机制，在服务器上，一定要放在防火墙之后**；
4. Memcached的**理论最大key长度为250字节**；
5. **单个item最大长度是1M**；
6. 连接数：
    6.1 并发数，最大最大是200（这个在Memcached的源码里面，需要对Memcached进行二次开发）；
    6.2 软连接数，最大长度是1024（这个在Memcached的源码里面，需要对Memcached进行二次开发）。
7. Memcached不提供冗余机制（但是XMemcached解决了，在servers中，`,`分隔备份，` `分隔不同节点）。

#### 3 Memcached使用建议
1. 基于**文本形式（包括网页、视频等）**的存储，Memcached目前效率最高；
2. 作为数据库前端；
    2.1 加快查询速度；
    2.2 减少数据库的访问次数；
3. 作为热点数据缓存；
4. 提升Web应用的速度；
5. 提高扩展性；
6. 缓存一些查询结果；
7. 强烈推荐使用多级缓存：
    7.1 本地缓存（EsCache或Oscache）
    7.2 内存应用（Memcached）
    
#### 4 Memcached使用场景
1. 分布式应用；
2. DB前端缓存；
3. 变化和查询频繁，又不需要入库；
4. 查询需求大，数据变更不频繁；